In [1]:
###########################################################
# Build the individual models
###########################################################

In [1]:
from pathlib import Path
from sklearn.utils import shuffle
from utils.confusion_matrix import cm as confusion_matrix 
from models.InceptionV1_3D import Inception_Inflated3d
from models.InceptionV3_3D import Inception_v3_Inflated3d
from tensorflow import keras
from numpy.random import default_rng
from tensorflow.keras import layers
from scipy import ndimage
from scipy.ndimage import zoom
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from utils.MulticlassAUC import MulticlassAUC
from os import path
from skimage.transform import resize
import tensorflow as tf
import os
import zipfile
import tensorflow as tf
import nibabel as nib
import matplotlib.pyplot as plt
import random
import multiprocessing as mp
import math
import numpy as np
import random

@tf.function
def rotate(volume):
    """Rotate the volume by a few degrees"""

    def scipy_rotate(volume):
        # define some rotation angles
        # pick angles at random
        angle=random.randint(-30,30) 
        #angle=random.randint(1,5) 
        # rotate volume
        volume = ndimage.rotate(volume, angle, reshape=False)
        volume[volume < 0] = 0
        volume[volume > 1] = 1
        return volume

     
    augmented_volume = tf.numpy_function(scipy_rotate, [volume], tf.float32)
    return augmented_volume


def translate(volume):
    def scipy_shift(volume):
        # shift volume
        shift_x = random.randint(-10,10) 
        shift_y = random.randint(-10,10) 
        shift_z = random.randint(-10,10)
        shift=[shift_x, shift_y, shift_z]
        volume = ndimage.shift(volume, shift)
        return volume

    augmented_volume = tf.numpy_function(scipy_shift, [volume], tf.float32)
    return augmented_volume
 
    
def train_preprocessing(volume, label):
    """Process training data by rotating and adding a channel."""
    
    k=np.random.randint(2, size=2)
    
    # Rotate volume
    if k[0]==1: 
        volume = rotate(volume)
        volume = tf.reshape(volume, [ct_scan_depth, ct_scan_width, ct_scan_height])
    
    # Translate volume
    if k[1]==1:
        volume = translate(volume)
        volume = tf.reshape(volume, [ct_scan_depth, ct_scan_width, ct_scan_height])
    
     
    volume = tf.reshape(volume, [ct_scan_depth, ct_scan_width, ct_scan_height])
    volume = tf.expand_dims(volume, axis=3)
    return volume, label



def test_and_validation_preprocessing(volume, label):
    """Process validation data by only adding a channel."""
    volume = tf.expand_dims(volume, axis=3)
    return volume, label



def training(cnn,
             ct_scan_width, 
             ct_scan_height, 
             ct_scan_depth,
             data_path,
             models_path,
             lr=0.00001, 
             dropout_prob=0.0, 
             epochs=100,
             batch_size=16,
             r_batch_size=16,
             n_folds=5):
    
    """
    cnn             : cnn architecture. Use 'iv1' to use Inception-V1 or 'iv3' to use Inception-V3
    ct_scan_width   : width of the CT scans 
    ct_scan_height  : height of the CT scans
    ct_scan_depth   : depth (number of slices) of the CT scans
    data_path       : path of the training/validation/test set for each fold
    models_path     : directory where the models will be saved. If the directory does not exist it is created
    lr              : learning rate 
    dropout_prob    : dropout value 
    epochs          : training epochs
    batch_size      : global batch size
    r_batch_size    : replica batch size
    n_folds         : number of folds for k-fold cross validation (default 5-fold cross validation)   
    """
    
    # CT scans size Width x Height x Depth
    ct_scan_width=ct_scan_width 
    ct_scan_height=ct_scan_height
    ct_scan_depth=ct_scan_depth
    
    # image channels
    num_channels = 1
    
    # training parameters
    epochs=epochs               # training epochs
    lr=lr                       # learning rate
    batch_size = batch_size     # global batch size
    r_batch_size = r_batch_size # replica batch size
    dropout_prob=dropout_prob   # dropout value for the last fully connected layer

    # number of folds 
    n_folds=n_folds

    idx = 0

    scores_acc = {'loss': np.zeros(n_folds), 
                  'categorical_accuracy': np.zeros(n_folds),
                  'specificity': np.zeros(n_folds),
                  'sensitivity': np.zeros(n_folds)
                 }

    scores_loss = {'loss': np.zeros(n_folds), 
                  'categorical_accuracy': np.zeros(n_folds),
                  'specificity': np.zeros(n_folds),
                  'sensitivity': np.zeros(n_folds)
                  }

    scores_test_lastepoch = {'loss': np.zeros(n_folds), 
                  'categorical_accuracy': np.zeros(n_folds),
                  'specificity': np.zeros(n_folds),
                  'sensitivity': np.zeros(n_folds)
                }

    f=0

    for i in range(n_folds): 

        if True:
            models_path=path.join(models_path, "fold_"+str(i+1) + path.sep)
            Path(models_path).mkdir(parents=True, exist_ok=True)


            # load preprocessed CLEAN-CC-CCII data for a given fold
            data_path=path.join(data_path, "fold_"+str(i+1) + path.sep)

            x_train = np.load(data_path+'x_train.npy')
            x_test = np.load(data_path+'x_test.npy')
            x_val = np.load(data_path+'x_val.npy')
            y_train = np.load(data_path+'y_train.npy')
            y_test = np.load(data_path+'y_test.npy')
            y_val = np.load(data_path+'y_val.npy')


            x_train.squeeze()
            x_test.squeeze()
            x_val.squeeze()

            x_train = np.swapaxes( x_train, 1, -1); x_train = np.swapaxes( x_train, -2, -1)
            x_test = np.swapaxes( x_test, 1, -1); x_test = np.swapaxes( x_test, -2, -1)
            x_val = np.swapaxes( x_val, 1, -1); x_val = np.swapaxes( x_val, -2, -1)


            print('Number of samples ' )
            print('Training set: %d chest CT scans' % (x_train.shape[0]))
            print('Test set: %d chest CT scans' % (x_test.shape[0]))
            print('Validation set: %d chest CT scans' % (x_val.shape[0]))

            
            
            model = None
            
            if cnn=='iv1':
                print("Using 3D Inception-V1")
                model = Inception_Inflated3d(include_top=False, 
                                                      input_shape=(ct_scan_depth, ct_scan_width, ct_scan_height, num_channels),
                                                      dropout_prob=dropout_prob, classes=3)
            elif cnn=='iv3':
                print("Using 3D Inception-V3")
                model = Inception_v3_Inflated3d(include_top=False, 
                                                  input_shape=(ct_scan_depth, ct_scan_width, ct_scan_height, num_channels),
                                                  dropout_prob=dropout_prob, classes=3)
            else:
                print("Please use 'iv1 for Inception-V1 or 'iv3' for Inception-V3")
                break
            # compile the model
            model.compile(loss=tf.keras.losses.categorical_crossentropy,
                       optimizer=tf.optimizers.Adam(lr=lr),
                        metrics=['categorical_accuracy',
                               MulticlassAUC(pos_label=0, name="AUC_Normal"),
                               MulticlassAUC(pos_label=1, name="AUC_NCP"),
                               MulticlassAUC(pos_label=2, name="AUC_CP")
                        ])


            print('\n**************************************************\n')
            print('Fold:', i+1)
            print('\n**************************************************\n')

            # Define data loaders.
            train_loader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
            validation_loader = tf.data.Dataset.from_tensor_slices((x_val, y_val)) 
            test_loader = tf.data.Dataset.from_tensor_slices((x_test, y_test))

            # Augment the on the fly during training.
            train_dataset = (
                train_loader.shuffle(len(x_train))
                .map(train_preprocessing, num_parallel_calls=batch_size)
                    .batch(batch_size)
                    .prefetch(r_batch_size)
            )
            # Only rescale.
            validation_dataset = (
                validation_loader.map(test_and_validation_preprocessing, num_parallel_calls=batch_size)
                    .batch(batch_size)
                    .prefetch(r_batch_size)
            )
            # Only rescale.
            test_dataset = (
                test_loader.map(test_and_validation_preprocessing, num_parallel_calls=batch_size)
                    .batch(batch_size)
                    .prefetch(r_batch_size)
            )

            early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_categorical_accuracy", min_delta=0.05, patience=15)
            
            log_cb=tf.keras.callbacks.TensorBoard(log_dir = "./tb_log_dir")

            checkpoint_filepath_1 = models_path+'best_model.h5'
            best_model_checkpoint_acc_cb = tf.keras.callbacks.ModelCheckpoint(
                filepath=checkpoint_filepath_1,
                save_weights_only=True,
                monitor='val_categorical_accuracy',
                mode='max',
                save_best_only=True
            )

            checkpoint_filepath_2 = models_path+'best_model_loss.h5'
            best_model_checkpoint_loss_cb = tf.keras.callbacks.ModelCheckpoint(
                filepath=checkpoint_filepath_2,
                save_weights_only=True,
                monitor='val_loss',
                mode='max',
                save_best_only=True
            )

            checkpoint_models_path = models_path+'{epoch:02d}-{categorical_accuracy:.2f}-{val_categorical_accuracy:.2f}.h5'
            all_models_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
                filepath=checkpoint_models_path,
                save_weights_only=True
            )

            history=model.fit(
                train_dataset,
                validation_data=validation_dataset,
                epochs=epochs,
                #shuffle=True,
                verbose=2,
                callbacks=[best_model_checkpoint_acc_cb,
                            best_model_checkpoint_loss_cb,
                            log_cb]
            )

            # Save model last epoch
            model.save_weights(models_path+'last_epoch.h5')

            #print('\nEvaluating TRAINING SET (last epoch) ')
            #model.evaluate(train_dataset, verbose=1)

            #print('\n**** Confusion Matrix ****')
            #y_pred = model.predict(train_dataset)
            #cm = confusion_matrix(y_train, y_pred)
            #print(cm)


            print('\nEvaluating VALIDATION SET (last epoch) ')
            model.evaluate(validation_dataset, verbose=1)

            print('\n**** Confusion Matrix ****')
            y_pred = model.predict(validation_dataset)
            cm = confusion_matrix(y_val, y_pred)
            print(cm)

            print('\nEvaluating TEST SET (last epoch)')

            score_test_lastepoch = model.evaluate(test_dataset, verbose=1)
            scores_test_lastepoch['loss'][idx]=score_test_lastepoch[0]
            scores_test_lastepoch['categorical_accuracy'][idx]=score_test_lastepoch[1]

            print('Test loss:', score_test_lastepoch[0])
            print('Test accuracy:', score_test_lastepoch[1])
            print('\n**** Confusion Matrix ****')

            y_pred = model.predict(test_dataset)
            cm = confusion_matrix(y_test, y_pred)
            print(cm)

            print('\nEvaluating TEST SET (best model)') 
            model.load_weights(checkpoint_filepath_1) 
            score_acc = model.evaluate(test_dataset, verbose=1)
            scores_acc['loss'][idx]=score_acc[0]
            scores_acc['categorical_accuracy'][idx]=score_acc[1]

            print('Test loss:', score_acc[0])
            print('Test accuracy:', score_acc[1])
            print('\n**** Confusion Matrix ****')
            y_pred = model.predict(test_dataset)
            cm = confusion_matrix(y_test, y_pred)
            print(cm)

            idx += 1
            

# start training for all folds
# replace data and models path with your directory


ct_scan_depth = 25; ct_scan_width = 256; ct_scan_height = 256;
dropout_prob=0.0
cnn='iv1'
data_path = "/home/pwrai/notebook/clean_cc_ccii_folds/"+str(ct_scan_depth)+"/" 

if cnn=='iv1':
    models_path = "/home/pwrai/notebook/my_models/"+str(ct_scan_depth)+"/Inception_v1/drop_"+str(dropout_prob).replace('.','')+"/fold_"+str(fold)+"/
elif cnn=='iv3':
    models_path = "/home/pwrai/notebook/my_models/"+str(ct_scan_depth)+"/Inception_v3/drop_"+str(dropout_prob).replace('.','')+"/fold_"+str(fold)+"/



training(cnn, ct_scan_width, ct_scan_height, ct_scan_depth, data_path, models_path, dropout_prob=dropout_prob)

KeyboardInterrupt: 

In [ ]:
#########################################
# 
# Soft-Voting Ensemble model
#
#########################################

In [ ]:
from itertools import product
from numpy import tensordot
from sklearn.metrics import accuracy_score
from models.InceptionV1_3D import Inception_Inflated3d
from models.InceptionV3_3D import Inception_v3_Inflated3d
from utils.confusion_matrix import cm as confusion_matrix 



def ensemble_predictions(members, weights, testX):
    # make predictions
    yhats = [model.predict(testX) for model in members]
    yhats = np.array(yhats)
    # weighted sum across ensemble members
    summed = tensordot(yhats, weights, axes=((0),(0)))
    # argmax across classes
    result = np.argmax(summed, axis=1)
    return result, summed 



# CNN architecture: use 'iv1' for Inception-V1 or 'iv3' for Inception-V3
cnn='iv1'

# replace with the FOLD that should be used to assess the ensemble model
fold=1

# parameters related to the CT scan sizes
ct_scan_depth = 25
ct_scan_width = 256
ct_scan_height = 256
num_channels = 1

# load test set
# replace with your directory
data_path = "/home/pwrai/notebook/clean_cc_ccii_folds/"+str(ct_scan_depth)+"/fold_"+fold+"/"
x_test = np.load(data_path+'x_test.npy')
y_test = np.load(data_path+'y_test.npy')

x_test = np.swapaxes( x_test, 1, -1); x_test = np.swapaxes( x_test, -2, -1)

test_loader = tf.data.Dataset.from_tensor_slices((x_test, y_test))

batch_size = 16 # global batch size
r_batch_size = 16 # replica batch size
test_dataset = (
                test_loader.map(test_and_validation_preprocessing, num_parallel_calls=batch_size)
                   .batch(batch_size)
                    .prefetch(r_batch_size)
                )

# load the individual models. For a given fold five models will be loaded
# replace the path of the models with your paths
# the loaded models (built for different dropout values) must refer to the same fold

model1=None
model2=None
model3=None
model4=None
model5=None

if cnn=='iv1':
    model1 = Inception_Inflated3d(include_top=False, 
                                input_shape=(ct_scan_depth, img_size, img_size, num_channels),
                                dropout_prob=0.0, classes=3)
    model1.load_weights("/home/pwrai/notebook/my_models/"+str(ct_scan_depth)+"/Inception_v1/drop_03/fold_"+str(fold)+"/best_model.h5")
    model2= Inception_Inflated3d(include_top=False, 
                                input_shape=(ct_scan_depth, img_size, img_size, num_channels),
                                dropout_prob=0.3, classes=3)
    model2.load_weights("/home/pwrai/notebook/my_models/"+str(ct_scan_depth)+"/Inception_v1/drop_03/fold_"+str(fold)+"/best_model.h5")

    model3 = Inception_Inflated3d(include_top=False, 
                                input_shape=(ct_scan_depth, img_size, img_size, num_channels),
                                dropout_prob=0.4, classes=3)
    model3.load_weights("/home/pwrai/notebook/my_models/"+str(ct_scan_depth)+"/Inception_v1/drop_04/fold_"+str(fold)+"/best_model.h5")       

    model4 = Inception_Inflated3d(include_top=False, 
                                input_shape=(ct_scan_depth, img_size, img_size, num_channels),
                                dropout_prob=0.5, classes=3)
    model4.load_weights("/home/pwrai/notebook/my_models/"+str(ct_scan_depth)+"/Inception_v1/drop_05/fold_"+str(fold)+"/best_model.h5")


    model5 = Inception_Inflated3d(include_top=False, 
                                input_shape=(ct_scan_depth, img_size, img_size, num_channels),
                                dropout_prob=0.6, classes=3)
    model5.load_weights("/home/pwrai/notebook/my_models/"+str(ct_scan_depth)+"/Inception_v1/drop_06/fold_"+str(fold)+"/best_model.h5")

elif cnn=='iv3':
    model1 = Inception_v3_Inflated3d(include_top=False, 
                                input_shape=(ct_scan_depth, img_size, img_size, num_channels),
                                dropout_prob=0.0, classes=3)
    model1.load_weights("/home/pwrai/notebook/my_models/"+str(ct_scan_depth)+"/Inception_v3/drop_00/fold_"+str(fold)+"/best_model.h5")

    model2= Inception_v3_Inflated3d(include_top=False, 
                                input_shape=(ct_scan_depth, img_size, img_size, num_channels),
                                dropout_prob=0.3, classes=3)
    model2.load_weights("/home/pwrai/notebook/my_models/"+str(ct_scan_depth)+"/Inception_v3/drop_03/fold_"+str(fold)+"/best_model.h5")

    model3 = Inception_v3_Inflated3d(include_top=False, 
                                input_shape=(ct_scan_depth, img_size, img_size, num_channels),
                                dropout_prob=0.4, classes=3)
    model3.load_weights("/home/pwrai/notebook/my_models/"+str(ct_scan_depth)+"/Inception_v3/drop_04/fold_"+str(fold)+"/best_model.h5")       

    model4 = Inception_v3_Inflated3d(include_top=False, 
                                input_shape=(ct_scan_depth, img_size, img_size, num_channels),
                                dropout_prob=0.5, classes=3)
    model4.load_weights("/home/pwrai/notebook/my_models/"+str(ct_scan_depth)+"/Inception_v3/drop_05/fold_"+str(fold)+"/best_model.h5")


    model5 = Inception_v3_Inflated3d(include_top=False, 
                                input_shape=(ct_scan_depth, img_size, img_size, num_channels),
                                dropout_prob=0.6, classes=3)
    model5.load_weights("/home/pwrai/notebook/my_models/"+str(ct_scan_depth)+"/Inception_v3/drop_06/fold_"+str(fold)+"/best_model.h5")
    
if model1!=None:
    members = [model1, model2, model3, model4, model5]
    weights = [1/5, 1/5, 1/5, 1/5, 1/5] 
    results, y_scores = ensemble_predictions(members, weights, test_dataset)


    # Assess results and print the confusion matrix
    y_pred=[]
    for i in results:
        if i==0: y_pred.append([1, 0 , 0])
        elif i==1: y_pred.append([0, 1, 0])
        elif i==2: y_pred.append([0, 0, 1])
    cm = confusion_matrix(y_test, y_pred)
    print(cm)

